In [ ]:
from osgeo import gdal
from math import floor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import struct

Python Documentation
https://gdal.org/api/python/osgeo.gdal.html
https://stackoverflow.com/questions/50191648/gis-geotiff-gdal-python-how-to-get-coordinates-from-pixel
https://gdal.org/programs/gdal2tiles.html
^ Use this last link with the os module to tlie the geotiff
https://www.youtube.com/watch?v=H5uQ85VXttg&list=PL4aUQR9L9RFp7kuu38hInDE-9ByueEMES&index=6
^ Or follow the ideas in this vid tutorial!!!

# Exploratory GDAL Manipulation

In [ ]:
# Opening the raster
raster = "./s1a-ew-grd-hv-20210304t163412-20210304t163512-036851-045567-002.tiff"
ds = gdal.Open(raster)

In [ ]:
# Obtaining GeoTIFF metadata
print("'ds' type", type(ds))
print("Projection: ", ds.GetProjection()) # get projection
print("Pixel width:", ds.RasterXSize)  # number of columns
print("Pixel Height:", ds.RasterYSize)  # number of rows
print("Band count:", ds.RasterCount)  # number of bands
print("GeoTransform", ds.GetGeoTransform()) # GeoTransform is a tuple of this sequence
# (topleftXCoord (xmin), pixelWidth (resolution in m), rowRotation(usually 0), topYCoord (ymax), columnRotation(usually 0), pixel height(in meters negative for north up images))
print("Projection", ds.GetProjection())

In [ ]:
# Reading the raster data
data_array = ds.GetRasterBand(1).ReadAsArray() # Indexing for raster bands starts at 1, not 0
data_array.shape
plt.figure(figsize=(10, 10))
plt.imshow(data_array) # Data array contains elevation data from the raster band. Is this always true?
plt.colorbar()
plt.show()

In [ ]:
# Rasterbands have a designated no-data-value, so we should figure out what it is
print("No data value:", ds.GetRasterBand(1).GetNoDataValue())

# Lets say that the value is None, we can always set one
ds.GetRasterBand(1).SetNoDataValue(0)
print("No data value:", ds.GetRasterBand(1).GetNoDataValue())

In [ ]:
# To close a raster, we just set the variable equal to none
ds = None

In [ ]:
## Here we're replicating the above process, but with the geoTiff that correlates to the coordinates of our CSV from the h5 file.
raster2 = "./20200223_154949_SCWA_HH_8bit_50m.tif"
ds2 = gdal.Open(raster2)
print(gdal.Info(ds2))

In [ ]:
# Reproject the test case into lat/long by specifying the dstSRS (ATL10 Data product says the lat/long is the EPSG coordinate system)
gdal.Warp("reprojected.tiff",raster2,dstSRS='EPSG:4326')
raster2 = "./reprojected.tiff"
ds2 = gdal.Open(raster2)

In [ ]:
print("Projection: ", ds2.GetProjection()) # get projection
print("Tiff width (pixels):", ds2.RasterXSize)  # number of columns
print("Tiff Height (pixels):", ds2.RasterYSize)  # number of rows
print("Band count:", ds2.RasterCount)  # number of bands
print("GeoTransform Matrix", ds2.GetGeoTransform())
print("Statistics:", ds2.GetRasterBand(1).GetStatistics(0,1))

data_array2 = ds2.GetRasterBand(1).ReadAsArray() # Indexing for raster bands starts at 1, not 0

plt.figure(figsize=(10, 10))
plt.imshow(data_array2) # Data array contains elevation data from the raster band. Is this always true?
plt.colorbar()
plt.show()

In [ ]:
print(data_array2.shape)

# Actual Script

In [ ]:
# Read in the CSV and store the values as a list of tuples
df = pd.read_csv("./TestCase1/3BeamCoordinates+Extra.csv")

# As of now, Latitude and Longitude are the coordinates of the GT1R strong beam.
# If 'Freeboard Height' is an invalid column name, it should be '1FBH'
# If we need more data, this dataframe and cooresponding ATL10 script should include the coordinate pairs extracted
# and recalculate for each beam so we can have a 3 to 6-fold increase our tiles.

# Split into the beam/ reading pairs
df1r = df[['gt1rLat', 'gt1rLon', 'gt1rFBH']].dropna().assign(beam="gt1r")
df1l = df[['gt1lLat', 'gt1lLon', 'gt1lFBH']].dropna().assign(beam="gt1l")
gt1r = list(zip(df1r['beam'], df1r['gt1rLat'], df1r['gt1rLon'], df1r['gt1rFBH']))
gt1l = list(zip(df1l['beam'], df1l['gt1lLat'], df1l['gt1lLon'], df1l['gt1lFBH']))
del df1r, df1l

df2r = df[['gt2rLat', 'gt2rLon', 'gt2rFBH']].dropna().assign(beam="gt2r")
df2l = df[['gt2lLat', 'gt2lLon', 'gt2lFBH']].dropna().assign(beam="gt2l")
gt2r = list(zip(df2r['beam'], df2r['gt2rLat'], df2r['gt2rLon'], df2r['gt2rFBH']))
gt2l = list(zip(df2l['beam'], df2l['gt2lLat'], df2l['gt2lLon'], df2l['gt2lFBH']))
del df2r, df2l

df3r = df[['gt3rLat', 'gt3rLon', 'gt3rFBH']].dropna().assign(beam="gt3r")
df3l = df[['gt3lLat', 'gt3lLon', 'gt3lFBH']].dropna().assign(beam="gt3l")
gt3r = list(zip(df3r['beam'], df3r['gt3rLat'], df3r['gt3rLon'], df3r['gt3rFBH']))
gt3l = list(zip(df3l['beam'], df3l['gt3lLat'], df3l['gt3lLon'], df3l['gt3lFBH']))
del df3r, df3l

atlasMeasurements = gt1r + gt1l + gt2r + gt2l + gt3r + gt3l
print(len(atlasMeasurements))
print(atlasMeasurements[0])
print(atlasMeasurements[-1])

In [ ]:
# Open, reproject, and normalize the SAR image raster
raster = "./TestCase1/s1a-ew-grd-hv-20210304t163412-20210304t163512-036851-045567-002.tiff"
ds = gdal.Open(raster)

# Reproject the test case into standard lat/long
gdal.Warp("reprojectedWSG84.tiff",raster,dstSRS='EPSG:4326')
raster2 = "./reprojectedWSG84.tiff"

# Open the new reprojection
ds2 = gdal.Open(raster2)

In [ ]:
# Reading and printing the raster data
band = ds2.GetRasterBand(1)
band_array = band.ReadAsArray() # Indexing for raster bands starts at 1, not 0
band_array.shape
xoffset, px_w, rot1, yoffset, rot2, px_h = ds2.GetGeoTransform() # The transformation matrix to get the coordinates from the picture

extent = [xoffset, yoffset, xoffset + px_w * ds2.RasterXSize, yoffset + px_h * ds2.RasterYSize]
print(extent)

plt.figure(figsize=(10, 10))
plt.imshow(band_array) # Data array contains elevation data from the raster band. Is this always true?
plt.colorbar()
plt.show()


In [ ]:
# Utility Functions
def getTileCenter(index: int):
    _ , lat, lon, fb_h = atlasMeasurements[index]
    pixel_x = (lon - xoffset) / px_w
    pixel_x = floor(pixel_x - 8.5)

    pixel_y = (lat - yoffset) / px_h
    pixel_y = floor(pixel_y - 8.5)

    calculatedCoord = (pixel_x, pixel_y)
    return calculatedCoord

def getFileDirectoryAndName(value: float, index: int, beam_name: str):
    path = "D:\Sentinel1\six-beam-directory"
    return path + "\\" + beam_name + str(index) + "-" + str(value) + ".tiff"

def isTileEmpty(pixelTuples: tuple):
    topLeftX, topLeftY = pixelTuples
    tile = band.ReadAsArray(topLeftX,topLeftY, 17, 17)
    if tile is not None and tile.any():
        sortList = sorted(tile[0])
        print([sortList[0], sortList[len(sortList)-1]])
        return True
    return False


In [ ]:
if rot1 == 0 and rot2 == 0:
    # We want to calculate the pixel, store the freeboard measurement, check the next coord, and write if it's not the same
    print("No need to worry about rotation, now converting")
    measuredVal = 0
    numRepeats = 0
    for i in range(len(atlasMeasurements) - 1):
        pixelCoords = getTileCenter(i)
        nextCoords = getTileCenter(i+1)
        beam_name,_,_, fb_h = atlasMeasurements[i]
        window = (*pixelCoords, 17, 17)
        
        # If the next point is at the same pixel coordinate, just update the running total
        if pixelCoords == nextCoords:
            measuredVal += fb_h
            numRepeats += 1
        else:
            # otherwise, tile the image using the average of the running total's freeboard measurement.
            if numRepeats == 0:
                tileName = getFileDirectoryAndName(fb_h, i, beam_name)
            else:
                measuredVal += fb_h
                numRepeats += 1
                tileName = getFileDirectoryAndName(measuredVal / numRepeats, i, beam_name)
            hasData = isTileEmpty(pixelCoords)
            if not hasData:
                print(i, "Has no data", pixelCoords)
            else:
                if i % 1000 == 0:
                    print(i, " - ", tileName)
                gdal.Translate(tileName, raster2, srcWin = window)
                measuredVal = 0
                numRepeats = 0
else:
    print("Cannot proceed, must rotate the image when finding pixels")
    


# Visualizing Beam Data

In [ ]:
df = pd.read_csv("./TestCase1/3BeamCoordinates+Extra.csv")
fbhr1 = df['gt1rFBH'].dropna().plot(kind='kde')
fbhr2 = df['gt2rFBH'].dropna().plot(kind='kde')
fbhr3 = df['gt3rFBH'].dropna().plot(kind='kde')

fbhl1 = df['gt1lFBH'].dropna().plot(kind='kde')
fbhl2 = df['gt2lFBH'].dropna().plot(kind='kde')
fbhl3 = df['gt3lFBH'].dropna().plot(kind='kde')


plt.legend(['gt1r', 'gt2r', 'gt3r', 'gt1l', 'gt2l', 'gt3l'])

In [ ]:
gt1r = df['gt1rFBH']
gt1l = df['gt1lFBH']
gt2r = df['gt2rFBH']
gt2l = df['gt2lFBH']
gt3r = df['gt3rFBH']
gt3l = df['gt3lFBH']

means = df[['gt1rFBH', 'gt1lFBH', 'gt2rFBH', 'gt2lFBH', 'gt3rFBH', 'gt3lFBH']].mean()

print(means.mean())

print(gt1r.min())
print(gt1r.max())
print(gt1r.mean())
print(gt1l.min())
print(gt1l.max())
print(gt1l.mean())


print(gt2r.min())
print(gt2r.max())
print(gt2r.mean())
print(gt2l.min())
print(gt2l.max())
print(gt2l.mean())

print(gt3r.min())
print(gt3r.max())
print(gt3r.mean())
print(gt3l.min())
print(gt3l.max())
print(gt3l.mean())